# Exemplos `RSelenium`

In [ ]:
# renv é um gerenciador de ambientes para R
renv::init()
renv::install()

Selecionando a porta de rede para abrir o Selenium Server

In [ ]:
port <- 4445L

Iniciando o Selenium Server e o Remote Driver

In [ ]:
# Inicia Selenium Server na porta escolhida
geckodriver <- wdman::selenium(port = port, verbose = FALSE)

# Inicia o Remote Driver
rem_dr <- RSelenium::remoteDriver(
  remoteServerAddr = "localhost",
  port = port,
  browserName = "firefox"
)

# Código para inciar o Remote Driver de forma 'headless'
# options <- list("moz:firefoxOptions" = list(args = list("--headless")))
# rem_dr <- RSelenium::remoteDriver(
#   remoteServerAddr = "localhost",
#   port = port,
#   browserName = "firefox",
#   extraCapabilities = options
# )

Abrindo uma janela do navegador

In [ ]:
rem_dr$open()

Maximizar a tela

In [ ]:
rem_dr$maxWindowSize()

## Exemplos de extracão de filmes com busca (https://www.imdb.com/)

Navegar até o site do IMDB

In [ ]:
rem_dr$navigate("https://www.imdb.com/")

Criar conexão de arquivo para salvar os dados

In [ ]:
file_conn <- file("../../data/rselenium/filmes.txt", open = "a")

In [ ]:
series <- c("WANDINHA","A CASA DO DRAGÃO", "THE BOYS")

header <- c("title", "score", "raiting", "popularity")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(serie in series) {
  search <- rem_dr$findElement(using = "xpath","//input[@id=\"suggestion-search\"]")
  search$sendKeysToElement(list(serie, key = "enter"))
  Sys.sleep(2)

  result <- rem_dr$findElement(using = "xpath", "//*[@id=\"__next\"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div[1]/a")
  result$clickElement()
  Sys.sleep(2)

  rating <- rem_dr$findElement(using = "css selector", ".sc-bde20123-1")
  rating <- nota$getElementText()

  score <- rem_dr$findElement(using = "css selector", ".sc-bde20123-3")
  score <- avaliacoes$getElementText()

  popularity <- rem_dr$findElement(using = "css selector", ".sc-5f7fb5b4-1")
  popularity <- popularity$getElementText()

  line <- c(serie, rating, score, popularity)
  writeLines(paste(line, collapse = ";"), file_conn, sep = "\n")

  rem_dr$goBack()
  rem_dr$goBack()
  Sys.sleep(2)
}

Fecha conexão com o arquivo

In [ ]:
close(file_conn)

Finaliza conexão do Remote Driver e Selenium Server

In [ ]:
rem_dr$close()
geckodriver$stop()